In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD

# **Loading the Dataset**

In [ ]:
train_dataset_path = '/kaggle/input/intel-image-classification/seg_train/seg_train/'
validation_dataset_path = '/kaggle/input/intel-image-classification/seg_test/seg_test/'

In [ ]:
IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 32

For preprocessing, I have used Image Data Generator for image augmentation which is technique to apply different kinds of transformations to the original image through shifting, flipping, rotating the image and all this is happening on the fly while your model is still in the training stage.
I have used this technique since it helps in **creating more variation** in the dataset and greatly **enhances model performance**. Moreover, it also **saves a lot of memory** since images are being loaded in batches

# Preprocessing and Normalizing Images from the Train Dataset. 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   shuffle=True)

# Preprocessing Images from the Test Dataset

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dataset_path,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                             class_mode='categorical',
                                                             shuffle=True)

# **Label Mappings**

In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(15, 12))
idx = 0

for i in range(2):
    for j in range(5):
        label = labels[np.argmax(train_generator[0][1][idx])]
        ax[i, j].set_title(f"{label}")
        ax[i, j].imshow(train_generator[0][0][idx][:, :, :])
        ax[i, j].axis("off")
        idx += 1

plt.tight_layout()
plt.suptitle("Sample Training Images", fontsize=21)
plt.show()

# **Simple Model Creation**

**Here a simple model is created with just 2 convolution layers with batch normalization using SGD with Momentum**

In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)))
model1.add(MaxPooling2D(2,2))
model1.add(BatchNormalization())
model1.add(Conv2D(32, (3, 3), activation = 'relu'))
model1.add(MaxPooling2D(2,2))
model1.add(BatchNormalization())
model1.add(Flatten())
model1.add(Dense(128, activation=tf.nn.relu))
model1.add(Dense(6, activation=tf.nn.softmax))
print(model1.summary())

# **Model1 Results**

In [ ]:
# Compile model
epochs = 10
lrate = 0.01
optimizer = SGD(learning_rate=lrate, momentum=0.9)
model1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

cnn = model1.fit(train_generator, epochs=10,validation_data=validation_generator,
                       verbose=2)

# **Evaluate Model1**

In [ ]:
metrics = list(cnn.history.keys())
loss_values = cnn.history[metrics[2]]
val_loss_values = cnn.history[metrics[0]]
acc_values = cnn.history[metrics[3]]
val_acc_values = cnn.history[metrics[1]]
print("\nFinal validation loss function is", val_loss_values[-1])
print("Final validation accuracy is", val_acc_values[-1])

In [ ]:
# summarize history for accuracy
plt.plot(cnn.history['accuracy'])
plt.plot(cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(cnn.history['loss'])
plt.plot(cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# *# Model 2 with Deep Network*

**In this model, there are deeper Conv layers where I have added the drop out method to optimize the model, modified the learning rate, passed more number of epochs and used the ADAM algorithm as an optimizer.**

In [ ]:
model = Sequential()
model.add(Conv2D(128, (5, 5), input_shape=(150,150, 3), padding='same', activation='relu', kernel_constraint=MaxNorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=MaxNorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=MaxNorm(3)))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.001
decay = lrate/epochs
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

In [ ]:
cnn2 = model.fit(train_generator, epochs=15,validation_data=validation_generator,
                       verbose=2)

**In the second model you can see that the accuracy has improved by 9%. I am also fitting a third model with more number of epochs to further enhance the accuracy of my model**

In [ ]:
metrics = list(cnn2.history.keys())
loss_values = cnn2.history[metrics[2]]
val_loss_values = cnn2.history[metrics[0]]
acc_values = cnn2.history[metrics[3]]
val_acc_values = cnn2.history[metrics[1]]
print("\nFinal validation loss function is", val_loss_values[-1])
print("Final validation accuracy is", val_acc_values[-1])

In [ ]:
# summarize history for accuracy
plt.plot(cnn2.history['accuracy'])
plt.plot(cnn2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(cnn2.history['loss'])
plt.plot(cnn2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# *Model 3 with More number of Epochs*

In [ ]:
cnn3 = model.fit(train_generator, epochs=25,validation_data=validation_generator,
                       verbose=2)

**The model accuracy has improved from 80 to 85%**

**# Plotting the Train and Test Scores**

In [ ]:
metrics = list(cnn3.history.keys())
loss_values = cnn3.history[metrics[2]]
val_loss_values = cnn3.history[metrics[0]]
acc_values = cnn3.history[metrics[3]]
val_acc_values = cnn3.history[metrics[1]]
print("\nFinal validation loss function is", val_loss_values[-1])
print("Final validation accuracy is", val_acc_values[-1])

In [ ]:
# summarize history for accuracy
plt.plot(cnn3.history['accuracy'])
plt.plot(cnn3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(cnn3.history['loss'])
plt.plot(cnn3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()